In [1]:
#performance test of the code

import time
import numpy as np
import random
from itertools import product
import decimal

import random
from decimal import Decimal, ROUND_HALF_UP

def round_to_two_decimal_places(value):
    #Rounds a value to two decimal places with ROUND_HALF_UP.
    return Decimal(value).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)



In [2]:
def generate_lower_bound_log(model, current_index, current_point, result, step_L , brought = 0):
    if current_index == len(model)-1:
        curr_point = current_point.copy()
        point = (curr_point + [curr_point[current_index-1] + model[current_index][0]])
        #print(point)
        result.append(point)
        return

    lower_bound, upper_bound = model[current_index]
    for i in np.arange(lower_bound + brought, upper_bound + brought + step_L, step_L):
        current_point.append(i)
        generate_lower_bound_log(model, current_index + 1, current_point, result , step_L, i )
        current_point.pop()

def generate_upper_bound_log(model, current_index, current_point, result, step_L, brought = 0):
    if current_index == len(model)-1:
        curr_point = current_point.copy()
        point = (curr_point + [curr_point[current_index-1] + model[current_index][1]])
        #print(point)
        result.append(point)
        return

    lower_bound, upper_bound = model[current_index]
    for i in np.arange(lower_bound + brought, upper_bound + brought+ step_L, step_L):
        current_point.append(i)
        generate_upper_bound_log(model, current_index + 1, current_point, result , step_L, i )
        current_point.pop()

def max_point(model):
    #returns the maximal point \sigma_M of the model 
    max_point = []
    for i in range(len(model)):
        if i == 0:
            max_point.append(model[i][1])
        else:
            max_point.append(model[i][1] + max_point[i-1])
    return max_point

def min_point(model):
    #returns the minimal point \sigma_m of the model 
    min_point = []
    for i in range(len(model)):
        if i == 0:
            min_point.append(model[i][0])
        else:
            min_point.append(model[i][0] + min_point[i-1])
    return min_point

def generate_random_example(dimension, cardinality, random_model):
    #generates a random example of problem given the parameters of dimension and cardinality, 
    #i.e. a sequential model with #trns = dimension and a random log accepted by the model with #traces = cardinality
    # if random_model = False, then the model will have all intervals as [0,1]
    if random_model == True:
        model = generate_random_model(dimension)
    else:
        model = [ [0,1] for i in range(dimension)]

    log = generate_random_log(model, cardinality)
    return log, model

def generate_random_model(dimension):
    #generates a random sequential model with the given dimension, with the extrema of the intervals as explained in the paper
    model = []
    for i in range(dimension):
        # Generate random bounds
        lower_bound = round(random.uniform(0, 5),2)
        upper_bound = lower_bound + round(random.uniform(0, 5),2)
        model.append((lower_bound, upper_bound))  
    print(model)

    return model

def generate_random_log(model, cardinality):
    #Generates a log with n = cardinality random (a random value taken from every interval) traces accepted by the model
    log = []

    for j in range(cardinality):
        ex = []
        for i in range(len(model)):
            if i == 0:
                # Generate a random value within the model bounds and round it to two decimal places
                ex.append(round(random.uniform(model[i][0], model[i][1]),2))
            else:
                # Add the previous value to the new random value, then round to two decimal places
                ex.append(round(random.uniform(model[i][0], model[i][1]) + ex[i-1],2))
        log.append(ex)

    return log

def bf_search_space(counter, dimension, brought, model, gamma, search_space,step, distance_type):
    #build the search space (set of "all" points accepted by the model) brute force
    #as the space is continuous, the parameter "step" determines the number of splits of the space of each dimension into equidistant points
    if counter-1 == dimension:
        # Base case: all loops have been executed
        search_space.append(gamma.copy())
        #print(gamma, search_space)
        return search_space
    else:
        lower = model[counter-1][0] + brought
        upper = model[counter-1][1] + brought
        for i in  np.linspace(lower, upper, step):
            gamma.append(i)  # Modify the values for the current loop
            if distance_type == 0:
                bf_search_space(counter + 1, dimension, i, model , gamma, search_space, step, distance_type)  #Recursively call the function for the next loop
            else:
                bf_search_space(counter + 1, dimension, 0, model , gamma, search_space, step, distance_type)
            gamma.pop()  # Remove the current loop's value

    if counter == 1:
        print('done')
        return search_space

def search_space_delay(model, step):
    #builds the search space when using the delay only distance
    #(discretized, still with the parameter step determining the number of splits of the space of each dimension into equidistant points)

    # Create a list of linspaces for each interval
    linspaces = [np.linspace(interval[0], interval[1], step) for interval in model]

    # Generate all combinations of points using itertools.product
    combinations = set(product(*linspaces))

    return combinations


def find_aa(model, L, step, distance_type):
    #finds all possible anti-alignments considering the discretized search space, and comparing every point brute force

    dimension = len(model)
    #print('dimension:', dimension)

    if distance_type == 0:
        search_space = bf_search_space(1, dimension, 0, model, [], [], step, distance_type)
    else:
        search_space = search_space_delay(model, step)

    print(model)
    print('Search space:', search_space)


    # Initialize variables
    max_distance = float('-inf')
    max_points = []

    # Convert the list L to a numpy array for efficient vector operations
    L_array = np.array(L)

    # Iterate over each point in the search space
    for gamma in search_space:
        # Convert gamma to a numpy array for efficient vector operations
        gamma_array = np.array(gamma)

        # Calculate the minimum distance for the current gamma
        distances = np.linalg.norm(L_array - gamma_array, ord=1, axis=1)  # Efficient L1 norm computation
        min_distance = np.min(distances)  # Minimum distance to any point in L

        # Update max_distance and max_points based on the minimum distance
        if min_distance > max_distance:
            max_distance = min_distance
            max_points = [gamma]
        elif min_distance == max_distance:
            max_points.append(gamma)

    return max_distance, max_points

def trace_back_sigmas_for_gammas(anti_alignments , L, model, max_distance):
    for gamma in anti_alignments:
        #print('Gamma:', gamma)
        for sigma in L:
            distance = sum([abs(gamma[i] - sigma[i]) for i in range(len(model))])
            if distance == max_distance:
                print('Sigma:', sigma, 'Gamma:' , gamma, 'Distance:', distance)
                #print('Distance:', sum([abs(gamma[i] - sigma[i]) for i in range(len(model))]))

def transform_log(L, model):
    #Transforms the log into the equivalent using flow functions (for delay only distance)
    for i, trace in enumerate(L):
        if len(model) == 1:
            trace_list = list([trace])
        else:
            trace_list = list(trace)

        old_trace = trace_list.copy()

        # Calculate deltas and ensure they are rounded to two decimal places
        for j in range(1, len(trace_list)):
            # Calculate the difference and round to two decimal places
            trace_list[j] = round(trace_list[j] - old_trace[j - 1],2)

        L[i] = trace_list

    return L

In [4]:
import pulp as plp
import time

def LPSolver(model, log, distance_type):
#Linear Programming solver with constraints and variables as described in the paper

    # Define the dimensionality of the problem
    n = len(model) #dimension of the space
    m = len(log) #number of points in L

    if distance_type == 1: #if delay-only distance is used, the log is transformed in equivalent flow functions traces
        start =  time.time()
        log = transform_log(log, model)
        elapsed_time = time.time() - start
        print(elapsed_time)

    # Create the LP problem
    prob = plp.LpProblem("Maximize minimal manhattan distance", plp.LpMaximize)

    # Define the decision variables
    x = plp.LpVariable.dicts("x", range(n), lowBound=0, cat = 'Continuous')

    #Define the variable to maximize
    z = plp.LpVariable("z", lowBound=0, cat = 'Continuous')

    # Define the constraints for x, i.e. the search space constraints
    if distance_type == 0:
        for i in range(n):
            if i == 0:
                prob += x[i] >= model[i][0]
                prob += x[i] <= model[i][1]
            else:
                prob += x[i] >= model[i][0] + x[i-1]
                prob += x[i] <= model[i][1] + x[i-1]
    else:
        for i in range(n):
            prob += x[i] >= model[i][0]
            prob += x[i] <= model[i][1]


    # Define the constraints for the absolute value of the difference between x and each point in L
    diff_plus = plp.LpVariable.dicts("diff_plus", (range(m), range(n)), lowBound=0, cat = 'Continuous')
    diff_minus = plp.LpVariable.dicts("diff_minus", (range(m), range(n)), lowBound=0, cat = 'Continuous')

    M = np.linalg.norm(np.array(max_point(model)) - np.array(min_point(model)), ord=1)
    print("M:",M)

    #binary variables
    b = plp.LpVariable.dicts("b", (range(m), range(n)), cat = 'Binary')

    for j in range(m): #for every sigma in L
        for i in range(n): #for every dimension
            #print(i)
            if len(model) == 1: #if the model is 1-dimensional
                prob += diff_plus[j][i] - diff_minus[j][i] == log[j] - x[i]
            else:
                prob += diff_plus[j][i] - diff_minus[j][i] == log[j][i] - x[i]

            prob += diff_plus[j][i] <= M*b[j][i]
            prob += diff_minus[j][i] <= M*(1-b[j][i])

        prob += z <= plp.lpSum([diff_plus[j][i] + diff_minus[j][i] for i in range(n)])

    #print(prob)

    # Define the objective function
    prob += z

    # Solve the problem
    prob.solve()

    # Print the results
    print("Status:", plp.LpStatus[prob.status])
    #print("Max Distance:", plp.value(prob.objective))

    optimal_point = [plp.value(x[i]) for i in range(n)]
    #print('Optimal Point:', optimal_point)

    return plp.value(prob.objective), optimal_point, prob.numVariables(), prob.numConstraints()



In [ ]:
def brute_force_solver(model, log, step, distance_type):
    #the log is to be transformed or not depending on whether the LP solver has already been used for the same log
    #in such case, the log is already transformed
    #if the BF solver is the only one used, this line has to be uncommented 
    """ 
    if distance_type == 1:
        log = transform_log(log, model)
    """
    max_distance, max_points = find_aa(model, log, step, distance_type)
    #print('Max Distance:', max_distance)
    #print('Optimal Point:', max_points[0])

    return max_distance, max_points

import json
import ast

def retrieve_example(cardinality, dimension):
  #retrieves the log and model from the JSON file given the cardinality and dimension of the problem
  #it retrieves it only for distance type 0 (stamp only), to retrieve the "original" log

  filepath = "/content/drive/MyDrive/lp_solver_logs.json"

  # Load JSON data from a file
  with open(filepath, "r") as f:
      json_data = json.load(f)

  key_pattern = f"{cardinality}-{dimension}-0"

  # Find the entry with the corresponding key
  for key, entry in json_data.items():
      if entry.get("Key") == key_pattern:
          # Get the Model and Log from the "Log Data"
          model = ast.literal_eval(entry.get("Log Data", {}).get("Model"))
          log = entry.get("Log Data", {}).get("Log")
          return model, log

  # If the key pattern is not found, return a message
  return {"error": "No data found for the given key pattern"}



In [ ]:
import time
import json
import pandas as pd
from itertools import product
import os

# Parameters to generate examples
cardinalities = [10,100]
dimensions = [10]
distance_types = [0,1]
#step = 10

# Generate list of pairs of cardinalities and dimensions
pairs = product(cardinalities, dimensions)

# File paths for storing results
json_filename = "/content/drive/MyDrive/BF_solver_logs.json"
csv_filename = "/content/drive/MyDrive/BF_solver_results.csv"
json_filename2 = "/content/drive/MyDrive/BF_solver_logs2.json"

def save_log_to_json(key, log_data, file_path):
    #Saves log and model to a JSON file with a unique identifier, along with the key
    
    # Convert the data to a serializable format
    log_data_serializable = {str(k): v for k, v in log_data.items()}

    # Generate a unique identifier for the log
    log_id = hash(json.dumps(log_data_serializable, sort_keys=True))

    # Structure for storing the log with the key
    log_entry = {
        "Key": key,
        "Log Data": log_data_serializable
    }

    # Create or update the JSON file
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            json.dump({}, f, indent=4)

    # Load existing data
    with open(file_path, 'r') as f:
        existing_data = json.load(f)

    # Add the new log with the unique identifier
    existing_data[str(log_id)] = log_entry

    # Write back the updated data
    with open(file_path, 'w') as f:
        json.dump(existing_data, f, indent=4)

    return log_id  # Return the unique identifier to reference this log

# Function to append to a CSV file
def append_results_to_csv(results_dict, file_path):
    #Appends data (result) to a CSV file. 

    df = pd.DataFrame([results_dict])
    file_exists = os.path.exists(file_path)

    # If the file does not exist or is empty, write headers
    if not file_exists or os.path.getsize(file_path) == 0:
        df.to_csv(file_path, mode='w', index=False, header=True, sep=';')
    else:
        df.to_csv(file_path, mode='a', index=False, header=False, sep=';')  # Append without headers


# Dictionary to store results
lp_results = {}

# LP-based experiments loop
for cardinality, dimension in pairs:

    # Generate\retrieve example 
    #model, log, = retrieve_example(cardinality, dimension)
    log, model = generate_random_example(dimension, cardinality, random_model = True)

    #print(model)

    for distance_type in distance_types:
        key = f"{cardinality}-{dimension}-{distance_type}"
        print(f"Cardinality: {cardinality}, Dimension: {dimension}, Distance Type: {distance_type}")

        # Save the log data to the JSON file and get the unique ID
        #The log is initially saved to the "backup" json file in order to have it saved even if the execution is interrupted
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename2) 

        start_time = time.time()
        max_dist_LP, opt_point_LP, numVars, numConstr = LPSolver(model, log, distance_type)
        elapsed_time_LP = time.time() - start_time

        start_time_BF = time.time()
        max_dist_BF, opt_point_BF = brute_force_solver(model, log, 5, distance_type)
        elapsed_time_BF = time.time() - start_time_BF

        # Save the log and model in json file
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename)

        # Store the results
        result_data = {
            'Key': key,
            'Dimension': dimension,
            'Cardinality': cardinality,
            'Distance Type': distance_type,
            #'Model': str(model),  # Ensure this is serializable
            #'Log': str(L),  # Ensure this is serializable
            'Elapsed Time (LP)': elapsed_time_LP,
            'Elapsed Time (BF)': elapsed_time_BF,
            #'Number of Variables (LP)': numVars,
            #'Number of Constraints (LP)': numConstr,
            'Solution (LP)': str(opt_point_LP),  # Ensure this is serializable
            'Solution (BF)': str(opt_point_BF),  # Ensure this is serializable
            'Max Distance (LP)': max_dist_LP,
            'Max Distance (BF)': max_dist_BF,
        }

        append_results_to_csv(result_data, csv_filename)


        # Display the results
        print(f"Elapsed Time (LP): {elapsed_time_LP:.6f} seconds")
        print(f"Elapsed Time (BF): {elapsed_time_BF:.6f} seconds")
        #print(f"Number of Variables (LP): {numVars}")
        #print(f"Number of Constraints (LP): {numConstr}")
        print(f"Max Distance (LP): {max_dist_LP}")
        print(f"Max Distance (BF): {max_dist_BF}")

        print()

        # Save the results to both JSON and CSV files
        #append_results_to_json(lp_results, json_filename)

print(f"LP-based results successfully appended {csv_filename}.")


NameError: name 'generate_random_example' is not defined